In [0]:
%%bash
pip uninstall -y google-cloud-dataflow
pip install --upgrade --force tensorflow_transform apache-beam[gcp]

In [0]:
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
from tensorflow_transform.beam import impl as beam_impl
import shutil
import os
import pandas as pd

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format
print(tf.__version__)

# Cloud Setup
This section is only required if running on cloud

In [0]:
REGION = 'asia-east1'
BUCKET = '{BUCKET}'
PROJECT = '{PROJECT}'
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [0]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

# Split Dataset

In [0]:
df = pd.read_csv('https://dl.dropboxusercontent.com/s/y7lm7aton223abm/spam.csv')[['v1', 'v2']]
df

In [0]:
RANDOM_SEED = 42
train = df.sample(frac=0.8, random_state=RANDOM_SEED)
eval = df.drop(train.index)
test = eval.sample(frac=0.5, random_state=RANDOM_SEED)
eval = eval.drop(test.index)

# Create Dataset using tf.transform

In [0]:
def is_valid(inputs):
  try:
    text = inputs['text']
    spam = inputs['spam']
    return (text in ('ham', 'spam')) and (len(text) > 0)
  except:
    return False
  
def preprocess_tft(inputs):
  module_url = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1"
  embed = hub.Module(module_url)
  result = {}
  result['text_embedding'] = embed([inputs])[0]
  result['spam'] = tf.identity(inputs['spam'])
  return result

def preprocess(on_cloud=False):
  import datetime
  import os
  import tempfile
  from apache_beam.io import tfrecordio
  from tensorflow_transform.coders import example_proto_coder
  from tensorflow_transform.tf_metadata import dataset_metadata
  from tensorflow_transform.tf_metadata import dataset_schema
  from tensorflow_transform.beam import tft_beam_io
  from tensorflow_transform.beam.tft_beam_io import transform_fn_io
  
  job_name = 